# DubTech Datathon: Drug Overdose in USA

**Authors: Clement, Calvin, Tilova**

---

ADD DESC

## Introduction  

Dataset link : [CDC](https://data.cdc.gov/NCHS/Drug-overdose-death-rates-by-drug-type-sex-age-rac/95ax-ymtc/about_data)

<a id = 'toc'></a>
    
## Table of Contents
---
1. [Data Wrangling](#wrangle)


**Importing Libraries**

In [1]:
import numpy as np
import pandas as pd

# data vis
import matplotlib.pyplot as plt
import seaborn as sns

<a id = 'wrangle'></a>

### 1. Data Wrangling
---
Loading the CSV file

In [2]:
unclean_df = pd.read_csv('../data/drug_overdose_data.csv')
# first 5 rows
unclean_df.head()

,INDICATOR,PANEL,PANEL_NUM,UNIT,UNIT_NUM,STUB_NAME,STUB_NAME_NUM,STUB_LABEL,STUB_LABEL_NUM,YEAR,YEAR_NUM,AGE,AGE_NUM,ESTIMATE,FLAG
0,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,1999,1,All ages,1.1,6.1,NaN
1,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2000,2,All ages,1.1,6.2,NaN
2,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2001,3,All ages,1.1,6.8,NaN
3,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2002,4,All ages,1.1,8.2,NaN
4,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2003,5,All ages,1.1,8.9,NaN


In [107]:
unclean_df.shape

(6228, 15)

In [108]:
unclean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6228 entries, 0 to 6227
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   INDICATOR       6228 non-null   object 
 1   PANEL           6228 non-null   object 
 2   PANEL_NUM       6228 non-null   int64  
 3   UNIT            6228 non-null   object 
 4   UNIT_NUM        6228 non-null   int64  
 5   STUB_NAME       6228 non-null   object 
 6   STUB_NAME_NUM   6228 non-null   int64  
 7   STUB_LABEL      6228 non-null   object 
 8   STUB_LABEL_NUM  6228 non-null   float64
 9   YEAR            6228 non-null   int64  
 10  YEAR_NUM        6228 non-null   int64  
 11  AGE             6228 non-null   object 
 12  AGE_NUM         6228 non-null   float64
 13  ESTIMATE        5117 non-null   float64
 14  FLAG            1111 non-null   object 
dtypes: float64(3), int64(5), object(7)
memory usage: 730.0+ KB


In [109]:
unclean_df.isna().sum()

INDICATOR            0
PANEL                0
PANEL_NUM            0
UNIT                 0
UNIT_NUM             0
STUB_NAME            0
STUB_NAME_NUM        0
STUB_LABEL           0
STUB_LABEL_NUM       0
YEAR                 0
YEAR_NUM             0
AGE                  0
AGE_NUM              0
ESTIMATE          1111
FLAG              5117
dtype: int64

In [3]:
unclean_df.duplicated().sum()

0

In [4]:
unclean_df['ESTIMATE'].isna().sum()/unclean_df.shape[0]

0.1783879254977521

In [5]:
unclean_df['FLAG'].value_counts()

FLAG
*    1111
Name: count, dtype: int64

From our findings, the number of null values in the `ESTIMATE` column is the same as the `FLAG` column labelled as `*`. It is indicating when the estimate is null as flag. If we drop the null values rows in the estimate column we do not need the flag column. So let's drop both.

In [6]:
unclean_df.drop(columns='FLAG', inplace=True)

In [7]:
unclean_df.dropna(subset='ESTIMATE', inplace=True)

In [8]:
unclean_df.isna().sum()

INDICATOR         0
PANEL             0
PANEL_NUM         0
UNIT              0
UNIT_NUM          0
STUB_NAME         0
STUB_NAME_NUM     0
STUB_LABEL        0
STUB_LABEL_NUM    0
YEAR              0
YEAR_NUM          0
AGE               0
AGE_NUM           0
ESTIMATE          0
dtype: int64

In [9]:
unclean_df['STUB_LABEL'].value_counts()

STUB_LABEL
All persons                                                                240
Female                                                                     240
Male                                                                       240
Male: Not Hispanic or Latino: White                                        126
Male: White                                                                126
Female: White                                                              126
Female: Not Hispanic or Latino: White                                      126
Male: Black or African American                                            125
Male: Not Hispanic or Latino: Black                                        125
Female: Not Hispanic or Latino: Black                                      124
Male: Hispanic or Latino: All races                                        124
Female: Black or African American                                          124
Female: Hispanic or Latino: All races    

Wow this is very messy, we have multiple rows indicating the same race for both genders. Let's try to simplify/clean this up!

We should have the following Races Male & Female:
- All Races
- American Indian or Alaska Native
- Black or African American
- White
- Asian or Pacific Islander

In [10]:
def relabel(current, new):
    assert isinstance(current, str), 'Current label must be a string'
    assert isinstance(new, str), 'New label must be a string'
    
    return np.where(unclean_df['STUB_LABEL']==current, new, unclean_df['STUB_LABEL'])

In [11]:
# Simplifying the labels
unclean_df['STUB_LABEL'] = relabel('Male: Hispanic or Latino: All races', 'Male: All Races')
unclean_df['STUB_LABEL'] = relabel('Female: Not Hispanic or Latino: American Indian or Alaska Native', 'Female: American Indian or Alaska Native')
unclean_df['STUB_LABEL'] = relabel('Female: Not Hispanic or Latino: Black', 'Female: Black or African American')
unclean_df['STUB_LABEL'] = relabel('Female: Not Hispanic or Latino: White', 'Female: White')
unclean_df['STUB_LABEL'] = relabel('Male: Not Hispanic or Latino: American Indian or Alaska Native', 'Male: American Indian or Alaska Native')
unclean_df['STUB_LABEL'] = relabel('Male: Not Hispanic or Latino: Black', 'Male: Black or African American')
unclean_df['STUB_LABEL'] = relabel('Male: Not Hispanic or Latino: White', 'Male: White')
unclean_df['STUB_LABEL'] = relabel('Female: Hispanic or Latino: All races', 'Female: All Races')
unclean_df['STUB_LABEL'] = relabel('Male: Not Hispanic or Latino: Asian or Pacific Islander', 'Male: Asian or Pacific Islander')
unclean_df['STUB_LABEL'] = relabel('Female: Not Hispanic or Latino: Asian or Pacific Islander', 'Female: Asian or Pacific Islander')
unclean_df['STUB_LABEL'] = relabel('Male: Not Hispanic or Latino: Asian', 'Male: Asian or Pacific Islander')
unclean_df['STUB_LABEL'] = relabel('Female: Not Hispanic or Latino: Asian', 'Female: Asian or Pacific Islander')
unclean_df['STUB_LABEL'] = relabel('Male: Not Hispanic or Latino: Native Hawaiian or Other Pacific Islander', 'Male: Asian or Pacific Islander')

In [12]:
# Sanity check
unclean_df['STUB_LABEL'].value_counts()

STUB_LABEL
Male: White                                 252
Female: White                               252
Male: Black or African American             250
Female: Black or African American           248
All persons                                 240
Female                                      240
Male                                        240
Male: American Indian or Alaska Native      185
Female: American Indian or Alaska Native    172
Male: Asian or Pacific Islander             142
Male: All Races                             124
Female: All Races                           122
Female: 35-44 years                         120
Male: 45-54 years                           120
Male: 35-44 years                           120
Male: 25-34 years                           120
Male: 15-24 years                           120
Female: 45-54 years                         120
25-34 years                                 120
55-64 years                                 120
45-54 years                  

Great this looks good!

In [13]:
unclean_df['STUB_LABEL_NUM'].value_counts()

STUB_LABEL_NUM
0.10    240
2.20    240
2.10    240
4.10    126
4.50    126
5.70    126
5.20    126
4.20    125
5.30    125
5.80    124
5.10    124
4.60    123
3.24    120
3.23    120
3.15    120
1.30    120
3.14    120
3.13    120
3.12    120
3.25    120
1.60    120
1.50    120
1.40    120
1.20    120
3.16    119
5.60    117
3.22    115
3.26    111
1.70    104
3.27     94
4.30     93
5.40     92
3.17     91
5.90     86
4.70     82
4.40     75
5.50     72
1.80     71
1.10     65
3.28     60
5.91     54
4.80     53
1.90     53
3.18     52
3.11     50
3.29     47
3.21     40
3.19     32
5.92      4
Name: count, dtype: int64

These look like categories but with integer datatype, let's look into one of these categories.

In [121]:
unclean_df[unclean_df['STUB_LABEL_NUM'] == 0.10]

,INDICATOR,PANEL,PANEL_NUM,UNIT,UNIT_NUM,STUB_NAME,STUB_NAME_NUM,STUB_LABEL,STUB_LABEL_NUM,YEAR,YEAR_NUM,AGE,AGE_NUM,ESTIMATE
0,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,1999,1,All ages,1.1,6.1
1,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2000,2,All ages,1.1,6.2
2,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2001,3,All ages,1.1,6.8
3,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2002,4,All ages,1.1,8.2
4,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2003,5,All ages,1.1,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6096,Drug overdose death rates,Drug overdose deaths involving methadone,3,"Deaths per 100,000 resident population, crude",2,Total,0,All persons,0.1,2018,20,All ages,1.1,0.9
6126,Drug overdose death rates,Drug overdose deaths involving other synthetic...,4,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2018,20,All ages,1.1,9.9
6147,Drug overdose death rates,Drug overdose deaths involving other synthetic...,4,"Deaths per 100,000 resident population, crude",2,Total,0,All persons,0.1,2018,20,All ages,1.1,9.6
6177,Drug overdose death rates,Drug overdose deaths involving heroin,5,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2018,20,All ages,1.1,4.7


Stub Label for category `0.10` is `All persons`. This is easy to comprehend but complicated to follow in the dataset given.

In [122]:
unclean_df['PANEL'].value_counts()

PANEL
All drug overdose deaths                                                         1037
Drug overdose deaths involving any opioid                                         997
Drug overdose deaths involving natural and semisynthetic opioids                  920
Drug overdose deaths involving other synthetic opioids (other than methadone)     736
Drug overdose deaths involving methadone                                          723
Drug overdose deaths involving heroin                                             704
Name: count, dtype: int64

In [123]:
unclean_df['PANEL_NUM'].value_counts()

PANEL_NUM
0    1037
1     997
2     920
4     736
3     723
5     704
Name: count, dtype: int64

The same goes for `PANEL` and `PANEL_NUM`. The numbers are categorized as the type of overdose for column PANEL. 

In [124]:
unclean_df.describe()

,PANEL_NUM,UNIT_NUM,STUB_NAME_NUM,STUB_LABEL_NUM,YEAR,YEAR_NUM,AGE_NUM,ESTIMATE
count,5117.000000,5117.000000,5117.00000,5117.000000,5117.000000,5117.000000,5117.000000,5117.000000
mean,2.241548,1.567520,2.98378,3.318476,2009.088919,11.088919,1.331409,4.743443
std,1.703687,0.495468,1.48050,1.555529,5.773726,5.773726,0.277185,6.424471
min,0.000000,1.000000,0.00000,0.100000,1999.000000,1.000000,1.100000,0.000000
25%,1.000000,1.000000,2.00000,2.100000,2004.000000,6.000000,1.100000,0.800000
50%,2.000000,2.000000,3.00000,3.220000,2009.000000,11.000000,1.100000,2.100000
75%,4.000000,2.000000,4.00000,4.600000,2014.000000,16.000000,1.600000,6.000000
max,5.000000,2.000000,5.00000,5.920000,2018.000000,20.000000,1.910000,54.300000


### Summary Analysis

1. **PANEL_NUM**
   - PANEL_NUM ranges from 0 to 5, with the majority of values concentrated between 1 and 4. The mean indicates that the average PANEL_NUM is around 2.24. Its PANEL indicates a `Drug overdose deaths involving natural and semisynthetic opioids`.

2. **UNIT_NUM**
   - UNIT_NUM has values of 1 or 2, with a mean of 1.57 indicating a relatively balanced distribution but slightly more frequent occurrences of the value 2. Its UNIT indicates a `Deaths per 100,000 resident population, crude`.

3. **STUB_NAME_NUM**
   - STUB_NAME_NUM ranges from 0 to 5, with a mean close to 3, indicating a fairly uniform distribution across its range. Its STUB NAME indicates a `Sex and age` category.

4. **STUB_LABEL_NUM**
   - STUB_LABEL_NUM ranges from 0.1 to 5.92, with a mean around 3.32, suggesting that values are well spread across its range. It is most frequently 0.10, 2.20 and 2.10, which are: `All persons`, `Female` and `Male`

5. **YEAR**
   - The data covers years from 1999 to 2018, with a mean year of around 2009.

6. **YEAR_NUM**
   - YEAR_NUM appears to be an encoded version of the YEAR column, possibly representing years since a starting point (e.g., 1999).

7. **AGE_NUM**
   - AGE_NUM has a narrow range from 1.1 to 1.91, with the majority of values around 1.1. Which are of `All ages`

8. **ESTIMATE**
   - ESTIMATE values range from 0 to 54.3, with a mean of 4.74. The high standard deviation indicates considerable variability in the data.

### Key Takeaways:
- **PANEL_NUM, STUB_NAME_NUM, and STUB_LABEL_NUM**: These columns are categorical identifiers, with values well distributed.
- **YEAR and YEAR_NUM**: These columns cover a period from 1999 to 2018.
- **AGE_NUM**: This column seems to have limited variability, indicating specific age categories.
- **ESTIMATE**: The primary variable of interest, showing considerable variation, which is crucial for modeling and prediction purposes.

### Next Steps:
1. **Data Preprocessing**: Ensure all categorical variables are encoded appropriately.
2. **Model Selection**: Given the dependent variable `ESTIMATE`.
3. **Train-Test Split**: Split the data ensuring temporal consistency.
4. **Modeling and Evaluation**: Implement and evaluate the models (time series model or regression model), ensuring they can showcase the demographic aspects accurately.

In [15]:
# Save the DataFrame as a CSV file
unclean_df.to_csv('../data/clean_drug_overdose_data.csv', index=False)

In [16]:
clean_data = pd.read_csv('../data/clean_drug_overdose_data.csv')

In [17]:
clean_data.head()

,INDICATOR,PANEL,PANEL_NUM,UNIT,UNIT_NUM,STUB_NAME,STUB_NAME_NUM,STUB_LABEL,STUB_LABEL_NUM,YEAR,YEAR_NUM,AGE,AGE_NUM,ESTIMATE
0,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,1999,1,All ages,1.1,6.1
1,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2000,2,All ages,1.1,6.2
2,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2001,3,All ages,1.1,6.8
3,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2002,4,All ages,1.1,8.2
4,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2003,5,All ages,1.1,8.9


### Preprocessing

create a new dataframe that has the stublabel encoding as - ml.csv

In [21]:
df_encoded = pd.get_dummies(clean_data, columns=['STUB_LABEL', 'PANEL'])

one_hot_columns = [col for col in df_encoded.columns if col not in clean_data.columns]

# Convert only the one-hot encoded columns to integers
df_encoded[one_hot_columns] = df_encoded[one_hot_columns].astype(int)

# Slicing out data for modelling 
df_encoded = df_encoded.iloc[:, 11:]

In [22]:
df_encoded.head()

,ESTIMATE,STUB_LABEL_15-24 years,STUB_LABEL_25-34 years,STUB_LABEL_35-44 years,STUB_LABEL_45-54 years,STUB_LABEL_55-64 years,STUB_LABEL_65-74 years,STUB_LABEL_75-84 years,STUB_LABEL_85 years and over,STUB_LABEL_All persons,...,STUB_LABEL_Male: Black or African American,STUB_LABEL_Male: Under 15 years,STUB_LABEL_Male: White,STUB_LABEL_Under 15 years,PANEL_All drug overdose deaths,PANEL_Drug overdose deaths involving any opioid,PANEL_Drug overdose deaths involving heroin,PANEL_Drug overdose deaths involving methadone,PANEL_Drug overdose deaths involving natural and semisynthetic opioids,PANEL_Drug overdose deaths involving other synthetic opioids (other than methadone)
0,6.1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,6.2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,6.8,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,8.2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,8.9,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [23]:
df_encoded.columns

Index(['ESTIMATE', 'STUB_LABEL_15-24 years', 'STUB_LABEL_25-34 years',
       'STUB_LABEL_35-44 years', 'STUB_LABEL_45-54 years',
       'STUB_LABEL_55-64 years', 'STUB_LABEL_65-74 years',
       'STUB_LABEL_75-84 years', 'STUB_LABEL_85 years and over',
       'STUB_LABEL_All persons', 'STUB_LABEL_Female',
       'STUB_LABEL_Female: 15-24 years', 'STUB_LABEL_Female: 25-34 years',
       'STUB_LABEL_Female: 35-44 years', 'STUB_LABEL_Female: 45-54 years',
       'STUB_LABEL_Female: 55-64 years', 'STUB_LABEL_Female: 65-74 years',
       'STUB_LABEL_Female: 75-84 years',
       'STUB_LABEL_Female: 85 years and over', 'STUB_LABEL_Female: All Races',
       'STUB_LABEL_Female: American Indian or Alaska Native',
       'STUB_LABEL_Female: Asian or Pacific Islander',
       'STUB_LABEL_Female: Black or African American',
       'STUB_LABEL_Female: Under 15 years', 'STUB_LABEL_Female: White',
       'STUB_LABEL_Male', 'STUB_LABEL_Male: 15-24 years',
       'STUB_LABEL_Male: 25-34 years', 'STUB_

In [24]:
df_encoded.to_csv('../data/encoded_drug_overdose_data.csv', index=False)